# Data Preparation: Video Game Sales


## Tujuan
Notebook ini menyiapkan dataset vgsales agar siap dipakai untuk:
- Struktur pasar.
- Preferensi regional.
- Siklus hidup platform.

Catatan metodologi:
- `Year` adalah tahun rilis, bukan time series penjualan.
- Analisis YoY tidak dipakai.
- Fokus pada cohort dan lifecycle.


----
## Load Data


In [54]:
import pandas as pd
import numpy as np
import os

df = pd.read_csv("https://raw.githubusercontent.com/AlvitoDwiP/video-game-sales/refs/heads/main/data/raw/vgsales.csv")
df.head()


,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Dataset memuat penjualan kumulatif per game dan region. Analisis menekankan struktur pasar dan perbandingan relatif.


### Pembulatan tahun rilis
Nilai `Year` dibulatkan agar konsisten sebagai tahun rilis (contoh 2008.0 menjadi 2008).


In [55]:
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["Year"] = df["Year"].round().astype("Int64")
df["Year"].head()


0    2006
1    1985
2    2008
3    2009
4    1996
Name: Year, dtype: Int64

### Drop baris tanpa identitas utama
Baris tanpa nama game atau platform tidak bisa dipakai untuk analisis entitas.


In [56]:
df = df.dropna(subset=["Name", "Platform"])
df.shape


(16598, 11)

Tidak ada baris terhapus; ukuran data tetap 16.598 x 11.


### Standardisasi kolom waktu
Kolom `Year` dibuat numerik agar cohort rilis bisa dihitung rapi.


In [57]:
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df["Year"] = df["Year"].round().astype("Int64")
df["Year"].isna().mean()


np.float64(0.016327268345583804)

Nilai `Year` yang hilang sekitar 1,63% dari seluruh baris.


### Penanganan kolom sales
Nilai sales yang kosong dianggap nol supaya agregasi tidak menghasilkan NaN.


In [58]:
sales_cols = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]
df[sales_cols] = df[sales_cols].fillna(0)
df[sales_cols].isna().sum()


NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64

Semua kolom sales (`NA_Sales`, `EU_Sales`, `JP_Sales`, `Other_Sales`) sudah bebas missing.


### Penanganan kategori
`Genre` dan `Publisher` yang kosong diisi `Unknown` agar kategori tetap konsisten.


In [59]:
df["Genre"] = df["Genre"].fillna("Unknown")
df["Publisher"] = df["Publisher"].fillna("Unknown")

df[["Genre", "Publisher"]].isna().sum()


Genre        0
Publisher    0
dtype: int64

Tidak ada missing tersisa pada `Genre` dan `Publisher`.


### Pembuatan total sales
`Total_Sales` dipakai sebagai acuan perbandingan lintas entitas.


In [60]:
df["Total_Sales"] = df[sales_cols].sum(axis=1)
df["Total_Sales"].describe()


count    16598.000000
mean         0.537164
std          1.555151
min          0.000000
25%          0.060000
50%          0.170000
75%          0.470000
max         82.740000
Name: Total_Sales, dtype: float64

`Total_Sales` condong ke ekor panjang: median 0,17 dan maksimum 82,74, jadi perbandingan relatif lebih masuk akal daripada rata-rata saja.


### Perhitungan share regional
Share dipakai untuk membandingkan preferensi pasar tanpa bias ukuran total.


In [61]:
df["Share_NA"] = df["NA_Sales"] / df["Total_Sales"].replace(0, np.nan)
df["Share_EU"] = df["EU_Sales"] / df["Total_Sales"].replace(0, np.nan)
df["Share_JP"] = df["JP_Sales"] / df["Total_Sales"].replace(0, np.nan)

df[["Share_NA", "Share_EU", "Share_JP"]] = df[["Share_NA", "Share_EU", "Share_JP"]].fillna(0)

df[["Share_NA", "Share_EU", "Share_JP"]].head()


,Share_NA,Share_EU,Share_JP
0,0.501450,0.350737,0.045564
1,0.722664,0.088966,0.169235
2,0.442367,0.359475,0.105777
3,0.477273,0.333636,0.099394
4,0.359146,0.283301,0.325685


Contoh 5 baris pertama menunjukkan pembagian proporsi penjualan per region (NA/EU/JP).


### Cohort rilis
Cohort dekade dipakai sebagai pengganti tren tahunan yang kurang valid.


In [62]:
df["decade_cohort"] = (df["Year"] // 10 * 10).astype("Int64")
df[["Year", "decade_cohort"]].dropna().head()


,Year,decade_cohort
0,2006,2000
1,1985,1980
2,2008,2000
3,2009,2000
4,1996,1990


Cohort dekade berhasil dibentuk (mis. 1980, 1990, 2000) dari kolom `Year`.


### Simpan output
Dataset ini menjadi input utama untuk analisis berikutnya.


In [63]:
project_root = Path.cwd().parent
output_dir = project_root / "data" / "processed"
output_dir.mkdir(parents=True, exist_ok=True)

output_path = output_dir / "vgsales_cleaned.csv"
df.to_csv(output_path, index=False)

output_path

PosixPath('/Users/a/Documents/video-game-sales/data/processed/vgsales_cleaned.csv')

File hasil tersimpan di `data/processed/vgsales.csv` pada root project.


**Insight**
- Dataset berisi 16.598 baris dan 11 kolom, cukup kuat untuk analisis struktur pasar lintas entitas.
- Baris tanpa `Name` dan `Platform` tidak ada, jadi identitas game dan platform relatif lengkap.
- Sekitar 1,63% nilai `Year` hilang; pendekatan berbasis cohort lebih aman daripada tren tahunan.
- Kolom penjualan regional sudah bersih dari missing, sehingga agregasi aman dilakukan.
- `Genre` dan `Publisher` kosong diisi `Unknown` agar kategori konsisten.
- Distribusi `Total_Sales` sangat miring: median 0,17 dan maksimum 82,74, jadi metrik relatif lebih informatif daripada rata-rata saja.
- Share regional (NA/EU/JP) membantu membandingkan preferensi pasar tanpa bias ukuran total.
- Cohort dekade dari `Year` cocok untuk analisis lifecycle dan pergeseran generasi platform.
- Output final tersedia di `data/processed/vgsales.csv` sebagai input analisis lanjutan.
